# Feature Selection on the 111011001 dataset

In this notebook we examine the result of dropping subsets of features that are linearly dependent or correlated to some degree.  We're specifically working on the dataset formed by dropping (31, 496, 524, 692, 917, 1299), since that resulted in the 3rd lowest validation error.

In [1]:
import itertools
import numpy as np
import pandas as pd

pd.set_option('display.precision',20)
pd.set_option('display.max_colwidth',100)

from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_predict, KFold, cross_val_score, \
                                    GridSearchCV, RandomizedSearchCV, ShuffleSplit 
from time import time
from scipy.stats import randint as sp_randint

import matplotlib.pylab as plt
from matplotlib.pylab import rcParams
from matplotlib import pyplot
rcParams['figure.figsize'] = 12, 4
%matplotlib inline

In [2]:
# def to compare goodness of fit on training set
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [3]:
# Cross-validation sets
kfold = KFold(n_splits=10, random_state=7)

# We are using LassoLarsCV as part of our metric
lr = linear_model.LassoLarsCV(verbose=False, max_iter=5000,precompute='auto', cv=kfold, max_n_alphas=1000, n_jobs=-1)

In [4]:
df = pd.read_csv("./input/train_tidy_111011001.csv")

In [5]:
ss = ShuffleSplit(n_splits=1, test_size=0.20, random_state=573)

In [6]:
X = df.values

In [7]:
for train_idx, validation_idx in ss.split(X):
    train_df = df.iloc[train_idx]
    validation_df = df.iloc[validation_idx]

We will establish a baseline by keeping all features.

In [8]:
y_validation = validation_df['SalePrice'].values
x_validation = validation_df.drop(['HouseId', 'SalePrice', 'GarageAge', 'GarageAgeLin'],axis=1).values
y_train = train_df['SalePrice'].values
x_train = train_df.drop(['HouseId', 'SalePrice', 'GarageAge', 'GarageAgeLin'],axis=1).values
lr.fit(x_train, y_train)
y_pred = lr.predict(x_validation)
baseline = rmse(y_validation, y_pred)
baseline

/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 255 iterations, i.e. alpha=1.730e-05, with an active set of 205 regressors, and the smallest cholesky pivot element being 2.220e-16
  ConvergenceWarning)
/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:334: ConvergenceWarning: Early stopping the lars path, as the residues are small and the current value of alpha is no longer well controlled. 348 iterations, alpha=2.092e-03, previous alpha=4.175e-06, with an active set of 257 regressors.
  ConvergenceWarning)
/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:334: ConvergenceWarning: Early stopping the lars path, as the residues are small and the current value of alpha is no longer well controlled. 346 iterations, alpha=1.119e-03, previous alpha=4.240e-06, with an active set of 249 regressors.
  ConvergenceWarning)
/usr/lib64/python2.7/

0.11777391463448175

## Features

We have a collection of Features, some of which were identified to be of potentially low-quality in predicting the response, others of which are known to be highly correlated with other Features.  We want to identify subsets of features that we can drop to improve the regression.

In [9]:
drop_cands = [
    'LotFrontage', 'LotArea', 'BsmtUnfSF', 'LowQualFinSF',
    'LogGrLivArea', 
    'GrLivArea', 'TotalHouseArea', 'LivArea', 'LivAreaWt', 'AllSizesSum', 'AllSizesSumLin', 'AreasSum',
    'X1stFlrSF','X1stLin', 'X2ndFlrSF', 'X2ndLin',
    'TotalBath', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
    'Age', 'AgeLin', 'RemodAgeLin','RemodAge',
    'MasVnrArea', 'MasVnrAreaLin',
    'DeckPorchLin','WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', 'X3SsnPorch', 'ScreenPorch'
]

In [10]:
corr_df = df[drop_cands].corr()

In [11]:
corr_df[corr_df > 0.75]

,LotFrontage,LotArea,BsmtUnfSF,LowQualFinSF,LogGrLivArea,GrLivArea,TotalHouseArea,LivArea,LivAreaWt,AllSizesSum,...,RemodAgeLin,RemodAge,MasVnrArea,MasVnrAreaLin,DeckPorchLin,WoodDeckSF,OpenPorchSF,EnclosedPorch,X3SsnPorch,ScreenPorch
LotFrontage,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LotArea,NaN,1.00000000000000000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.98598983721606414665,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BsmtUnfSF,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LowQualFinSF,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LogGrLivArea,NaN,NaN,NaN,NaN,1.00000000000000000000,0.97412505575337704489,0.86212699291813443825,0.76682978644455090933,0.92065495638440875137,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GrLivArea,NaN,NaN,NaN,NaN,0.97412505575337704489,1.00000000000000000000,0.83209394002082814890,0.76709010302272995219,0.91315684498877447428,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TotalHouseArea,NaN,NaN,NaN,NaN,0.86212699291813443825,0.83209394002082814890,1.00000000000000000000,0.80386530803955380531,0.87672471508279026242,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LivArea,NaN,NaN,NaN,NaN,0.76682978644455090933,0.76709010302272995219,0.80386530803955380531,1.00000000000000000000,0.94604349127603937220,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LivAreaWt,NaN,NaN,NaN,NaN,0.92065495638440875137,0.91315684498877447428,0.87672471508279026242,0.94604349127603937220,1.00000000000000000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AllSizesSum,NaN,0.98598983721606414665,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00000000000000000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We'll restrict our drop set to the highly correlated features to make this more readable,

In [12]:
drop_cands = [
    'LotArea', 'LogGrLivArea', 
    'GrLivArea', 'TotalHouseArea', 'LivArea', 'LivAreaWt', 'AllSizesSum', 'AreasSum',
    'X1stFlrSF','X1stLin', 'TotalBath', 'FullBath', 'MasVnrArea', 'MasVnrAreaLin'
]

In [13]:
corr_df = df[drop_cands].corr()

In [14]:
corr_df[corr_df > 0.75]

,LotArea,LogGrLivArea,GrLivArea,TotalHouseArea,LivArea,LivAreaWt,AllSizesSum,AreasSum,X1stFlrSF,X1stLin,TotalBath,FullBath,MasVnrArea,MasVnrAreaLin
LotArea,1.00000000000000000000,NaN,NaN,NaN,NaN,NaN,0.98598983721606414665,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LogGrLivArea,NaN,1.00000000000000000000,0.97412505575337704489,0.86212699291813443825,0.76682978644455090933,0.92065495638440875137,NaN,0.83372960559711761785,NaN,NaN,NaN,NaN,NaN,NaN
GrLivArea,NaN,0.97412505575337704489,1.00000000000000000000,0.83209394002082814890,0.76709010302272995219,0.91315684498877447428,NaN,0.82003704646568897907,NaN,NaN,NaN,NaN,NaN,NaN
TotalHouseArea,NaN,0.86212699291813443825,0.83209394002082814890,1.00000000000000000000,0.80386530803955380531,0.87672471508279026242,NaN,0.93136002100177917029,NaN,NaN,NaN,NaN,NaN,NaN
LivArea,NaN,0.76682978644455090933,0.76709010302272995219,0.80386530803955380531,1.00000000000000000000,0.94604349127603937220,NaN,0.79673347849910070018,NaN,NaN,NaN,NaN,NaN,NaN
LivAreaWt,NaN,0.92065495638440875137,0.91315684498877447428,0.87672471508279026242,0.94604349127603937220,1.00000000000000000000,NaN,0.86294195014918295694,NaN,NaN,NaN,NaN,NaN,NaN
AllSizesSum,0.98598983721606414665,NaN,NaN,NaN,NaN,NaN,1.00000000000000000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AreasSum,NaN,0.83372960559711761785,0.82003704646568897907,0.93136002100177917029,0.79673347849910070018,0.86294195014918295694,NaN,1.00000000000000000000,NaN,NaN,NaN,NaN,NaN,NaN
X1stFlrSF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00000000000000000000,0.97831482212891651695,NaN,NaN,NaN,NaN
X1stLin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.97831482212891651695,1.00000000000000000000,NaN,NaN,NaN,NaN


Let's first compare the pairs.

In [15]:
drop_cands = ['LotArea', 'AllSizesSum']

In [16]:
col_drop_results_df = pd.DataFrame(dtype = 'float64')
count = 0
for L in range(0, len(drop_cands)+1):
        for subset in itertools.combinations(drop_cands, L):
                drop_cols = list(subset)
                col_drop_results_df.loc[count, 'Dropped'] = str(subset)
                x_train = train_df.drop(['HouseId', 'SalePrice', 'GarageAge', 'GarageAgeLin'] + drop_cols,axis=1).values
                x_validation = validation_df.drop(['HouseId', 'SalePrice', 'GarageAge', 'GarageAgeLin'] + drop_cols,axis=1).values
                lr.fit(x_train, y_train)
                y_pred = lr.predict(x_validation)
                error = rmse(y_validation, y_pred)
                col_drop_results_df.loc[count, 'RMSE'] = error
                col_drop_results_df.loc[count, 'Diff from Base'] = error - baseline                
                count += 1
output_df = col_drop_results_df.sort_values(['RMSE'])

/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 255 iterations, i.e. alpha=1.730e-05, with an active set of 205 regressors, and the smallest cholesky pivot element being 2.220e-16
  ConvergenceWarning)
/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 315 iterations, i.e. alpha=7.559e-06, with an active set of 237 regressors, and the smallest cholesky pivot element being 1.490e-08
  ConvergenceWarning)
/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:334: ConvergenceWarning: Early stopping the lars path, as the residues are small and the current value of alpha is no longer well controlled. 316 iterations, alpha=7.493e-06, previous alpha=7.477e-06, with an active set of 237 regressors.
  ConvergenceWarning)
/usr/lib64/python2.7/site-packages/skl

In [17]:
output_df

,Dropped,RMSE,Diff from Base
2,"('AllSizesSum',)",0.11776657220690905925,-0.00000734242757269299
1,"('LotArea',)",0.11777344125652022822,-0.00000047337796152402
0,(),0.11777391463448175224,0.00000000000000000000
3,"('LotArea', 'AllSizesSum')",0.11788532068255033747,0.00011140604806858523


We should keep these features.

In [18]:
drop_cands = ['X1stFlrSF','X1stLin']

In [19]:
col_drop_results_df = pd.DataFrame(dtype = 'float64')
count = 0
for L in range(0, len(drop_cands)+1):
        for subset in itertools.combinations(drop_cands, L):
                drop_cols = list(subset)
                col_drop_results_df.loc[count, 'Dropped'] = str(subset)
                x_train = train_df.drop(['HouseId', 'SalePrice', 'GarageAge', 'GarageAgeLin'] + drop_cols,axis=1).values
                x_validation = validation_df.drop(['HouseId', 'SalePrice', 'GarageAge', 'GarageAgeLin'] + drop_cols,axis=1).values
                lr.fit(x_train, y_train)
                y_pred = lr.predict(x_validation)
                error = rmse(y_validation, y_pred)
                col_drop_results_df.loc[count, 'RMSE'] = error
                col_drop_results_df.loc[count, 'Diff from Base'] = error - baseline                
                count += 1
output_df = col_drop_results_df.sort_values(['RMSE'])

/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 255 iterations, i.e. alpha=1.730e-05, with an active set of 205 regressors, and the smallest cholesky pivot element being 2.220e-16
  ConvergenceWarning)
/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 315 iterations, i.e. alpha=7.559e-06, with an active set of 237 regressors, and the smallest cholesky pivot element being 1.490e-08
  ConvergenceWarning)
/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:334: ConvergenceWarning: Early stopping the lars path, as the residues are small and the current value of alpha is no longer well controlled. 316 iterations, alpha=7.493e-06, previous alpha=7.477e-06, with an active set of 237 regressors.
  ConvergenceWarning)
/usr/lib64/python2.7/site-packages/skl

In [20]:
output_df

,Dropped,RMSE,Diff from Base
1,"('X1stFlrSF',)",0.11777341114251470289,-0.00000050349196704935
0,(),0.11777391463448175224,0.00000000000000000000
3,"('X1stFlrSF', 'X1stLin')",0.11791828616210375813,0.00014437152762200589
2,"('X1stLin',)",0.11802057879280447583,0.00024666415832272359


In [21]:
drop_cands = ['TotalBath', 'FullBath']

In [22]:
col_drop_results_df = pd.DataFrame(dtype = 'float64')
count = 0
for L in range(0, len(drop_cands)+1):
        for subset in itertools.combinations(drop_cands, L):
                drop_cols = list(subset)
                col_drop_results_df.loc[count, 'Dropped'] = str(subset)
                x_train = train_df.drop(['HouseId', 'SalePrice', 'GarageAge', 'GarageAgeLin'] + drop_cols,axis=1).values
                x_validation = validation_df.drop(['HouseId', 'SalePrice', 'GarageAge', 'GarageAgeLin'] + drop_cols,axis=1).values
                lr.fit(x_train, y_train)
                y_pred = lr.predict(x_validation)
                error = rmse(y_validation, y_pred)
                col_drop_results_df.loc[count, 'RMSE'] = error
                col_drop_results_df.loc[count, 'Diff from Base'] = error - baseline                
                count += 1
output_df = col_drop_results_df.sort_values(['RMSE'])

/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 255 iterations, i.e. alpha=1.730e-05, with an active set of 205 regressors, and the smallest cholesky pivot element being 2.220e-16
  ConvergenceWarning)
/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:334: ConvergenceWarning: Early stopping the lars path, as the residues are small and the current value of alpha is no longer well controlled. 346 iterations, alpha=1.119e-03, previous alpha=4.240e-06, with an active set of 249 regressors.
  ConvergenceWarning)
/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:334: ConvergenceWarning: Early stopping the lars path, as the residues are small and the current value of alpha is no longer well controlled. 348 iterations, alpha=2.092e-03, previous alpha=4.175e-06, with an active set of 257 regressors.
  ConvergenceWarning)
/usr/lib64/python2.7/

In [23]:
output_df

,Dropped,RMSE,Diff from Base
2,"('FullBath',)",0.11777344125652031148,-0.00000047337796144076
0,(),0.11777391463448175224,0.00000000000000000000
3,"('TotalBath', 'FullBath')",0.11844710426539675763,0.00067318963091500539
1,"('TotalBath',)",0.11845866438140841004,0.00068474974692665780


The gain by dropping both of these isn't big enough to consider dropping a predictor that we'd consider strong by hedonic reasoning.

In [24]:
drop_cands = ['MasVnrArea', 'MasVnrAreaLin']

In [25]:
col_drop_results_df = pd.DataFrame(dtype = 'float64')
count = 0
for L in range(0, len(drop_cands)+1):
        for subset in itertools.combinations(drop_cands, L):
                drop_cols = list(subset)
                col_drop_results_df.loc[count, 'Dropped'] = str(subset)
                x_train = train_df.drop(['HouseId', 'SalePrice', 'GarageAge', 'GarageAgeLin'] + drop_cols,axis=1).values
                x_validation = validation_df.drop(['HouseId', 'SalePrice', 'GarageAge', 'GarageAgeLin'] + drop_cols,axis=1).values
                lr.fit(x_train, y_train)
                y_pred = lr.predict(x_validation)
                error = rmse(y_validation, y_pred)
                col_drop_results_df.loc[count, 'RMSE'] = error
                col_drop_results_df.loc[count, 'Diff from Base'] = error - baseline                
                count += 1
output_df = col_drop_results_df.sort_values(['RMSE'])

/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 255 iterations, i.e. alpha=1.730e-05, with an active set of 205 regressors, and the smallest cholesky pivot element being 2.220e-16
  ConvergenceWarning)
/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:334: ConvergenceWarning: Early stopping the lars path, as the residues are small and the current value of alpha is no longer well controlled. 346 iterations, alpha=1.119e-03, previous alpha=4.240e-06, with an active set of 249 regressors.
  ConvergenceWarning)
/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 315 iterations, i.e. alpha=7.559e-06, with an active set of 237 regressors, and the smallest cholesky pivot element being 1.490e-08
  ConvergenceWarning)
/usr/lib64/python2.7/site-packages/skl

In [26]:
output_df

,Dropped,RMSE,Diff from Base
1,"('MasVnrArea',)",0.11774645755050452967,-0.00002745708397722257
2,"('MasVnrAreaLin',)",0.11777341114251492493,-0.00000050349196682731
0,(),0.11777391463448175224,0.00000000000000000000
3,"('MasVnrArea', 'MasVnrAreaLin')",0.11781890874551098625,0.00004499411102923401


In [27]:
drop_cands = ['LogGrLivArea', 'GrLivArea', 'TotalHouseArea', 'LivArea', 'LivAreaWt', 'AreasSum']

In [28]:
col_drop_results_df = pd.DataFrame(dtype = 'float64')
count = 0
for L in range(0, len(drop_cands)+1):
        for subset in itertools.combinations(drop_cands, L):
                drop_cols = list(subset)
                col_drop_results_df.loc[count, 'Dropped'] = str(subset)
                x_train = train_df.drop(['HouseId', 'SalePrice', 'GarageAge', 'GarageAgeLin'] + drop_cols,axis=1).values
                x_validation = validation_df.drop(['HouseId', 'SalePrice', 'GarageAge', 'GarageAgeLin'] + drop_cols,axis=1).values
                lr.fit(x_train, y_train)
                y_pred = lr.predict(x_validation)
                error = rmse(y_validation, y_pred)
                col_drop_results_df.loc[count, 'RMSE'] = error
                col_drop_results_df.loc[count, 'Diff from Base'] = error - baseline                
                count += 1
output_df = col_drop_results_df.sort_values(['RMSE'])

/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 255 iterations, i.e. alpha=1.730e-05, with an active set of 205 regressors, and the smallest cholesky pivot element being 2.220e-16
  ConvergenceWarning)
/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:334: ConvergenceWarning: Early stopping the lars path, as the residues are small and the current value of alpha is no longer well controlled. 346 iterations, alpha=1.119e-03, previous alpha=4.240e-06, with an active set of 249 regressors.
  ConvergenceWarning)
/usr/lib64/python2.7/site-packages/sklearn/linear_model/least_angle.py:334: ConvergenceWarning: Early stopping the lars path, as the residues are small and the current value of alpha is no longer well controlled. 348 iterations, alpha=2.092e-03, previous alpha=4.175e-06, with an active set of 257 regressors.
  ConvergenceWarning)
/usr/lib64/python2.7/

In [29]:
output_df

,Dropped,RMSE,Diff from Base
44,"('LogGrLivArea', 'GrLivArea', 'TotalHouseArea', 'AreasSum')",0.11584740756896885761,-0.00192650706551289463
58,"('LogGrLivArea', 'GrLivArea', 'TotalHouseArea', 'LivArea', 'AreasSum')",0.11609183723881348616,-0.00168207739566826608
46,"('LogGrLivArea', 'GrLivArea', 'LivArea', 'AreasSum')",0.11622059444764813729,-0.00155332018683361495
59,"('LogGrLivArea', 'GrLivArea', 'TotalHouseArea', 'LivAreaWt', 'AreasSum')",0.11623034092542024187,-0.00154357370906151037
25,"('LogGrLivArea', 'GrLivArea', 'AreasSum')",0.11623505029948129341,-0.00153886433500045883
47,"('LogGrLivArea', 'GrLivArea', 'LivAreaWt', 'AreasSum')",0.11633757429912623682,-0.00143634033535551542
15,"('GrLivArea', 'AreasSum')",0.11659159189640913579,-0.00118232273807261645
36,"('GrLivArea', 'LivArea', 'AreasSum')",0.11659578339835725835,-0.00117813123612449389
50,"('LogGrLivArea', 'TotalHouseArea', 'LivAreaWt', 'AreasSum')",0.11668531058552206181,-0.00108860404895969043
18,"('TotalHouseArea', 'AreasSum')",0.11669604460212060215,-0.00107787003236115009
